In [1]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
import numpy as np
import time
import config as cfg

begin = time.strftime('%Y-%m-%d_%H-%M-%S')

LOGDIR = './save'

tf.logging.set_verbosity(tf.logging.ERROR)

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

start_learning_rate = 0.5e-3  # 1e-3
adjust_learning_rate = 1e-5

#onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=4)
onehot_labels = tf.one_hot(indices=tf.reshape(tf.cast(model.y_, tf.int32),[-1]), depth=cfg.NUM_KEYS)

loss = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

loss_val = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=model.y)
# loss_val = tf.nn.softmax_cross_entropy_with_logits_v2( labels=onehot_labels, logits=model.y)
# train_step = tf.train.AdamOptimizer(start_learning_rate).minimize(loss)

# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
# tf.scalar_summary("loss", loss)
tf.summary.scalar("loss", loss)

tf.summary.scalar("loss_val", loss_val)

# merge all summaries into a single op
# merged_summary_op = tf.merge_all_summaries()
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = tf.train.SaverDef.V2)
# saver.restore(sess, "save/model.ckpt")

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

#epochs = 30
epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images/batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        # print('Each item in xs : shape is...')
        # print(xs[0].shape)
        
        # train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.7})
        loss_value = loss.eval(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0})
        
        # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))
        print("Epoch: %d, Step: %d, Loss: %g" % (epoch, i, loss_value))

        if i % 10 == 0:
            xs_val, ys_val = driving_data.LoadValBatch(batch_size)
            # xs, ys = driving_data.LoadTrainBatch(batch_size)
            loss_val = loss.eval(feed_dict={model.x:xs_val, model.y_: ys_val, model.keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss_val: %g" % (epoch, i, loss_val))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
            print("Model saved in file: %s" % filename)


correct_prediction = tf.equal(tf.argmax(onehot_labels, 1), tf.argmax(model.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Train Accuracy:', sess.run(accuracy, feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}))
print('Validation Accuracy:', sess.run(accuracy, feed_dict={model.x: xs_val, model.y_: ys_val, model.keep_prob: 1.0}))

end = time.strftime('%Y-%m-%d_%H-%M-%S')
print('begin: ', begin)
print('end: ', end)

print("Run the command line:\n",
      "--> tensorboard --logdir=./logs --port=6006",
      "\nThen open http://0.0.0.0:6006/ into your web browser")

# os.system("sudo -s shutdown -h now")




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
model read
Epoch: 0, Step: 0, Loss: 1.78272
Epoch: 0, Step: 0, Loss_val: 1.78105
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 1, Loss: 1.78194
Epoch: 0, Step: 2, Loss: 1.75499
Epoch: 0, Step: 3, Loss: 1.76923
Epoch: 0, Step: 4, Loss: 1.77612
Epoch: 0, Step: 5, Loss: 1.782
Epoch: 0, Step: 6, Loss: 1.78231
Epoch: 0, Step: 7, Loss: 1.78026
Epoch: 0, Step: 8, Loss: 1.78081
Epoch: 0, Step: 9, Loss: 1.77332
Epoch: 0, Step: 10, Loss: 1.7687
Epoch: 0, Step: 10, Loss_val: 1.77556
Epoch: 0, Step: 11, Loss: 1.78412
Epoch: 0, Step: 12, Loss: 1.77839
Epoch: 0, Step: 13, Loss: 1.78856
Epoch: 0, Step: 14, Loss: 1.77534
Epoch: 0, Step: 15, Loss: 1.76724
Epoch: 0, Step: 16, Loss: 1.76665
Epoch: 0, Step: 17, Loss: 1.76771
Epoch: 0, Step: 18, Loss: 1.77999
Epoch: 0, Step: 19, Loss: 1.77341
Epoch: 0, Step: 20, Loss: 1.76252
Epoch: 0, Step: 20, Loss_val: 1.76883
Epoch: 0, Step: 21,

Epoch: 0, Step: 193, Loss: 0.428547
Epoch: 0, Step: 194, Loss: 0.353954
Epoch: 0, Step: 195, Loss: 0.391213
Epoch: 0, Step: 196, Loss: 0.492827
Epoch: 0, Step: 197, Loss: 0.522558
Epoch: 0, Step: 198, Loss: 0.473975
Epoch: 0, Step: 199, Loss: 0.368399
Epoch: 0, Step: 200, Loss: 0.419478
Epoch: 0, Step: 200, Loss_val: 0.375157
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 201, Loss: 0.433074
Epoch: 0, Step: 202, Loss: 0.427499
Epoch: 0, Step: 203, Loss: 0.408894
Epoch: 0, Step: 204, Loss: 0.508583
Epoch: 0, Step: 205, Loss: 0.403329
Epoch: 0, Step: 206, Loss: 0.404436
Epoch: 0, Step: 207, Loss: 0.402876
Epoch: 0, Step: 208, Loss: 0.343666
Epoch: 0, Step: 209, Loss: 0.425423
Epoch: 0, Step: 210, Loss: 0.422463
Epoch: 0, Step: 210, Loss_val: 0.550642
Epoch: 0, Step: 211, Loss: 0.405928
Epoch: 0, Step: 212, Loss: 0.512629
Epoch: 0, Step: 213, Loss: 0.495057
Epoch: 0, Step: 214, Loss: 0.348593
Epoch: 0, Step: 215, Loss: 0.48772
Epoch: 0, Step: 216, Loss: 0.546322
Epoch: 0, Step: 21

Epoch: 0, Step: 397, Loss: 0.438871
Epoch: 0, Step: 398, Loss: 0.359942
Epoch: 0, Step: 399, Loss: 0.597101
Epoch: 0, Step: 400, Loss: 0.473875
Epoch: 0, Step: 400, Loss_val: 0.379947
Model saved in file: ./save/model.ckpt
Epoch: 0, Step: 401, Loss: 0.371447
Epoch: 1, Step: 0, Loss: 0.374031
Epoch: 1, Step: 0, Loss_val: 0.376815
Model saved in file: ./save/model.ckpt
Epoch: 1, Step: 1, Loss: 0.302822
Epoch: 1, Step: 2, Loss: 0.313091
Epoch: 1, Step: 3, Loss: 0.265689
Epoch: 1, Step: 4, Loss: 0.27128
Epoch: 1, Step: 5, Loss: 0.457353
Epoch: 1, Step: 6, Loss: 0.404653
Epoch: 1, Step: 7, Loss: 0.315306
Epoch: 1, Step: 8, Loss: 0.425893
Epoch: 1, Step: 9, Loss: 0.465055
Epoch: 1, Step: 10, Loss: 0.272169
Epoch: 1, Step: 10, Loss_val: 0.341795
Epoch: 1, Step: 11, Loss: 0.240531
Epoch: 1, Step: 12, Loss: 0.363705
Epoch: 1, Step: 13, Loss: 0.411957
Epoch: 1, Step: 14, Loss: 0.425605
Epoch: 1, Step: 15, Loss: 0.325979
Epoch: 1, Step: 16, Loss: 0.271038
Epoch: 1, Step: 17, Loss: 0.454958
Epoch:

Epoch: 1, Step: 200, Loss_val: 0.398896
Model saved in file: ./save/model.ckpt
Epoch: 1, Step: 201, Loss: 0.436662
Epoch: 1, Step: 202, Loss: 0.332144
Epoch: 1, Step: 203, Loss: 0.305155
Epoch: 1, Step: 204, Loss: 0.299289
Epoch: 1, Step: 205, Loss: 0.262565
Epoch: 1, Step: 206, Loss: 0.220463
Epoch: 1, Step: 207, Loss: 0.277655
Epoch: 1, Step: 208, Loss: 0.372041
Epoch: 1, Step: 209, Loss: 0.28362
Epoch: 1, Step: 210, Loss: 0.324858
Epoch: 1, Step: 210, Loss_val: 0.285799
Epoch: 1, Step: 211, Loss: 0.364161
Epoch: 1, Step: 212, Loss: 0.455251
Epoch: 1, Step: 213, Loss: 0.323823
Epoch: 1, Step: 214, Loss: 0.25594
Epoch: 1, Step: 215, Loss: 0.28291
Epoch: 1, Step: 216, Loss: 0.272163
Epoch: 1, Step: 217, Loss: 0.35525
Epoch: 1, Step: 218, Loss: 0.343125
Epoch: 1, Step: 219, Loss: 0.305452
Epoch: 1, Step: 220, Loss: 0.247047
Epoch: 1, Step: 220, Loss_val: 0.298882
Epoch: 1, Step: 221, Loss: 0.362398
Epoch: 1, Step: 222, Loss: 0.281644
Epoch: 1, Step: 223, Loss: 0.280639
Epoch: 1, Step: 2

Epoch: 2, Step: 0, Loss_val: 0.35386
Model saved in file: ./save/model.ckpt
Epoch: 2, Step: 1, Loss: 0.275314
Epoch: 2, Step: 2, Loss: 0.2772
Epoch: 2, Step: 3, Loss: 0.314197
Epoch: 2, Step: 4, Loss: 0.231333
Epoch: 2, Step: 5, Loss: 0.240692
Epoch: 2, Step: 6, Loss: 0.414028
Epoch: 2, Step: 7, Loss: 0.27511
Epoch: 2, Step: 8, Loss: 0.234257
Epoch: 2, Step: 9, Loss: 0.342554
Epoch: 2, Step: 10, Loss: 0.24513
Epoch: 2, Step: 10, Loss_val: 0.345429
Epoch: 2, Step: 11, Loss: 0.303237
Epoch: 2, Step: 12, Loss: 0.239816
Epoch: 2, Step: 13, Loss: 0.35717
Epoch: 2, Step: 14, Loss: 0.341918
Epoch: 2, Step: 15, Loss: 0.352563
Epoch: 2, Step: 16, Loss: 0.409874
Epoch: 2, Step: 17, Loss: 0.362154
Epoch: 2, Step: 18, Loss: 0.276025
Epoch: 2, Step: 19, Loss: 0.449013
Epoch: 2, Step: 20, Loss: 0.249632
Epoch: 2, Step: 20, Loss_val: 0.32638
Epoch: 2, Step: 21, Loss: 0.195284
Epoch: 2, Step: 22, Loss: 0.288695
Epoch: 2, Step: 23, Loss: 0.284448
Epoch: 2, Step: 24, Loss: 0.290277
Epoch: 2, Step: 25, L

...

Epoch: 27, Step: 360, Loss_val: 0.126232
Epoch: 27, Step: 361, Loss: 0.0949716
Epoch: 27, Step: 362, Loss: 0.0398859
Epoch: 27, Step: 363, Loss: 0.128132
Epoch: 27, Step: 364, Loss: 0.0830726
Epoch: 27, Step: 365, Loss: 0.0527638
Epoch: 27, Step: 366, Loss: 0.105196
Epoch: 27, Step: 367, Loss: 0.0830442
Epoch: 27, Step: 368, Loss: 0.0747853
Epoch: 27, Step: 369, Loss: 0.0420156
Epoch: 27, Step: 370, Loss: 0.07862
Epoch: 27, Step: 370, Loss_val: 0.131331
Epoch: 27, Step: 371, Loss: 0.0661027
Epoch: 27, Step: 372, Loss: 0.0484242
Epoch: 27, Step: 373, Loss: 0.0740706
Epoch: 27, Step: 374, Loss: 0.0297748
Epoch: 27, Step: 375, Loss: 0.0754893
Epoch: 27, Step: 376, Loss: 0.0517703
Epoch: 27, Step: 377, Loss: 0.0799496
Epoch: 27, Step: 378, Loss: 0.0747377
Epoch: 27, Step: 379, Loss: 0.0306051
Epoch: 27, Step: 380, Loss: 0.0310937
Epoch: 27, Step: 380, Loss_val: 0.325035
Epoch: 27, Step: 381, Loss: 0.0614371
Epoch: 27, Step: 382, Loss: 0.091216
Epoch: 27, Step: 383, Loss: 0.0524961
Epoch: 2

Epoch: 28, Step: 154, Loss: 0.0994201
Epoch: 28, Step: 155, Loss: 0.0780492
Epoch: 28, Step: 156, Loss: 0.0808614
Epoch: 28, Step: 157, Loss: 0.115537
Epoch: 28, Step: 158, Loss: 0.0923753
Epoch: 28, Step: 159, Loss: 0.0981221
Epoch: 28, Step: 160, Loss: 0.0643026
Epoch: 28, Step: 160, Loss_val: 0.0918572
Epoch: 28, Step: 161, Loss: 0.070818
Epoch: 28, Step: 162, Loss: 0.0426634
Epoch: 28, Step: 163, Loss: 0.195409
Epoch: 28, Step: 164, Loss: 0.0759922
Epoch: 28, Step: 165, Loss: 0.074758
Epoch: 28, Step: 166, Loss: 0.0797028
Epoch: 28, Step: 167, Loss: 0.0531146
Epoch: 28, Step: 168, Loss: 0.136283
Epoch: 28, Step: 169, Loss: 0.0402088
Epoch: 28, Step: 170, Loss: 0.105279
Epoch: 28, Step: 170, Loss_val: 0.0484693
Epoch: 28, Step: 171, Loss: 0.0330991
Epoch: 28, Step: 172, Loss: 0.0336697
Epoch: 28, Step: 173, Loss: 0.0467378
Epoch: 28, Step: 174, Loss: 0.0820475
Epoch: 28, Step: 175, Loss: 0.0446011
Epoch: 28, Step: 176, Loss: 0.0545802
Epoch: 28, Step: 177, Loss: 0.0505593
Epoch: 28,

Epoch: 28, Step: 349, Loss: 0.0777622
Epoch: 28, Step: 350, Loss: 0.0789243
Epoch: 28, Step: 350, Loss_val: 0.120056
Epoch: 28, Step: 351, Loss: 0.0709442
Epoch: 28, Step: 352, Loss: 0.0775034
Epoch: 28, Step: 353, Loss: 0.0489838
Epoch: 28, Step: 354, Loss: 0.0377355
Epoch: 28, Step: 355, Loss: 0.0961229
Epoch: 28, Step: 356, Loss: 0.0330196
Epoch: 28, Step: 357, Loss: 0.080118
Epoch: 28, Step: 358, Loss: 0.0552854
Epoch: 28, Step: 359, Loss: 0.0683115
Epoch: 28, Step: 360, Loss: 0.0496493
Epoch: 28, Step: 360, Loss_val: 0.0755131
Epoch: 28, Step: 361, Loss: 0.0905251
Epoch: 28, Step: 362, Loss: 0.0811833
Epoch: 28, Step: 363, Loss: 0.0489687
Epoch: 28, Step: 364, Loss: 0.114513
Epoch: 28, Step: 365, Loss: 0.0672918
Epoch: 28, Step: 366, Loss: 0.097958
Epoch: 28, Step: 367, Loss: 0.059113
Epoch: 28, Step: 368, Loss: 0.0639169
Epoch: 28, Step: 369, Loss: 0.0697723
Epoch: 28, Step: 370, Loss: 0.0432454
Epoch: 28, Step: 370, Loss_val: 0.115947
Epoch: 28, Step: 371, Loss: 0.0710343
Epoch:

Epoch: 29, Step: 142, Loss: 0.054685
Epoch: 29, Step: 143, Loss: 0.0871705
Epoch: 29, Step: 144, Loss: 0.057671
Epoch: 29, Step: 145, Loss: 0.0894124
Epoch: 29, Step: 146, Loss: 0.0597314
Epoch: 29, Step: 147, Loss: 0.101213
Epoch: 29, Step: 148, Loss: 0.0442656
Epoch: 29, Step: 149, Loss: 0.0626836
Epoch: 29, Step: 150, Loss: 0.067614
Epoch: 29, Step: 150, Loss_val: 0.0979522
Epoch: 29, Step: 151, Loss: 0.0477493
Epoch: 29, Step: 152, Loss: 0.021613
Epoch: 29, Step: 153, Loss: 0.0668711
Epoch: 29, Step: 154, Loss: 0.0353192
Epoch: 29, Step: 155, Loss: 0.0564847
Epoch: 29, Step: 156, Loss: 0.0484431
Epoch: 29, Step: 157, Loss: 0.0500653
Epoch: 29, Step: 158, Loss: 0.100555
Epoch: 29, Step: 159, Loss: 0.0456238
Epoch: 29, Step: 160, Loss: 0.0408449
Epoch: 29, Step: 160, Loss_val: 0.136452
Epoch: 29, Step: 161, Loss: 0.0509165
Epoch: 29, Step: 162, Loss: 0.0923318
Epoch: 29, Step: 163, Loss: 0.0273769
Epoch: 29, Step: 164, Loss: 0.0626001
Epoch: 29, Step: 165, Loss: 0.12251
Epoch: 29, St

Epoch: 29, Step: 337, Loss: 0.103898
Epoch: 29, Step: 338, Loss: 0.0516404
Epoch: 29, Step: 339, Loss: 0.0195002
Epoch: 29, Step: 340, Loss: 0.0833051
Epoch: 29, Step: 340, Loss_val: 0.055493
Epoch: 29, Step: 341, Loss: 0.12603
Epoch: 29, Step: 342, Loss: 0.104068
Epoch: 29, Step: 343, Loss: 0.0400589
Epoch: 29, Step: 344, Loss: 0.0598479
Epoch: 29, Step: 345, Loss: 0.122368
Epoch: 29, Step: 346, Loss: 0.0660868
Epoch: 29, Step: 347, Loss: 0.0236735
Epoch: 29, Step: 348, Loss: 0.0655445
Epoch: 29, Step: 349, Loss: 0.23964
Epoch: 29, Step: 350, Loss: 0.0615765
Epoch: 29, Step: 350, Loss_val: 0.0619303
Epoch: 29, Step: 351, Loss: 0.0252142
Epoch: 29, Step: 352, Loss: 0.0260932
Epoch: 29, Step: 353, Loss: 0.0615551
Epoch: 29, Step: 354, Loss: 0.0986424
Epoch: 29, Step: 355, Loss: 0.0584644
Epoch: 29, Step: 356, Loss: 0.0388231
Epoch: 29, Step: 357, Loss: 0.111001
Epoch: 29, Step: 358, Loss: 0.0543159
Epoch: 29, Step: 359, Loss: 0.0467858
Epoch: 29, Step: 360, Loss: 0.0733397
Epoch: 29, St